## SQLITE

In [1]:
import sqlite3

In [3]:
sqlite3.version

'2.6.0'

In [4]:
sqlite3.sqlite_version

'3.32.3'

In [5]:
con = sqlite3.connect('C:/Users/park1/Desktop/sqlite/study.db',isolation_level=None)   ## isolation_level=None  ===> 자동으로 commit해주기

In [6]:
cur = con.cursor()

In [7]:
cur.executescript(''' 
    CREATE TABLE artists(id INTEGER PRIMARY KEY , name TEXT NOT NULL);
    CREATE TABLE album(id INTEGER PRIMARY KEY, name TEXT NOT NULL, fk INTEGER NOT NULL);
    CREATE TABLE track(id INTEGER PRIMARY KEY, name TEXT NOT NULL, rating INTEGER, length INTEGER, afk INTEGER NOT NULL, gfk INTEGER NOT NULL);
    CREATE TABLE genre(id INTEGER PRIMARY KEY, name TEXT NOT NULL)''')

In [8]:
cur.execute("INSERT INTO artists (name) values ('beenzino')")

In [11]:
cur.lastrowid

1

In [12]:
cur.execute("INSERT INTO artists(name) values ('paulkim')")

In [13]:
cur.lastrowid

2

In [16]:
cur.execute("INSERT INTO album (name, fk) values(?, (SELECT id FROM artists WHERE name='beenzino'))",(['aquaman']))

In [18]:
cur.execute('INSERT INTO album (name, fk) values(?,(SELECT id FROM artists WHERE name="paulkim"))',(['rain']))

In [20]:
cur.execute("SELECT * from album")
cur.fetchall()

[(1, 'aquaman', 1), (2, 'rain', 2)]

In [34]:
Genre = [{'name':'rap'},{'name':'ballad'}]

In [35]:
cur.executemany("INSERT INTO genre (name) VALUES(:name)", Genre)

In [39]:
track=[{'name':'been1st', 'fk1':'beenzino', 'fk2':'rap'},
       {'name':'paul1st', 'fk1':'paulkim', 'fk2':'ballad'}
      ]

cur.executemany("INSERT INTO TRACK (name, afk, gfk) VALUES ((:name),(SELECT id FROM album WHERE name=:fk1),(SELECT id FROM genre WHERE name=:fk2))", track)


IntegrityError: NOT NULL constraint failed: track.afk

## ORM

- SQLAlchemy 

In [30]:
! pip install sqlalchemy

In [40]:
import sqlalchemy

In [41]:
sqlalchemy.__version__

'1.3.18'

In [42]:
from sqlalchemy import create_engine

In [47]:
engine = create_engine("sqlite:///alchemy_study.db", echo=True)  ## 실제 데이터베이스와 통신

In [48]:
print(engine)

Engine(sqlite:///alchemy_study.db)


In [49]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [51]:
metadata = MetaData() ## class들의 instance를 관리

- table 생성

In [52]:
obj_users = Table('T_users', metadata, 
                 Column('pk', Integer, primary_key=True),
                 Column('name', String))


In [53]:
obj_addresses =Table('T_addresses',metadata,
                    Column('pk', Integer, primary_key=True),
                    Column('name', String),
                    Column('fk',None, ForeignKey('T_users.pk')))

In [55]:
for i, _ in enumerate(metadata.tables):
    print(i+1, _)

1 T_users
2 T_addresses


In [56]:
metadata.create_all(engine)

2021-07-05 20:44:49,833 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-05 20:44:49,835 INFO sqlalchemy.engine.base.Engine ()
2021-07-05 20:44:49,837 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-05 20:44:49,841 INFO sqlalchemy.engine.base.Engine ()
2021-07-05 20:44:49,843 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("T_users")
2021-07-05 20:44:49,845 INFO sqlalchemy.engine.base.Engine ()
2021-07-05 20:44:49,848 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("T_users")
2021-07-05 20:44:49,850 INFO sqlalchemy.engine.base.Engine ()
2021-07-05 20:44:49,852 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("T_addresses")
2021-07-05 20:44:49,853 INFO sqlalchemy.engine.base.Engine ()
2021-07-05 20:44:49,855 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("T_addresses")
2021-07-05 20:44:49,856 INFO sqlalchemy.engine.base.Engine ()
2021-07-05

In [57]:
print(obj_users.insert().values(pk=1, name='park').compile())

INSERT INTO "T_users" (pk, name) VALUES (:pk, :name)


In [62]:
print(obj_users.params)

<bound method Immutable.params of Table('T_users', MetaData(bind=None), Column('pk', Integer(), table=<T_users>, primary_key=True, nullable=False), Column('name', String(), table=<T_users>), schema=None)>


In [63]:
con = engine.connect()

In [64]:
con.execute(obj_users.insert(), {'pk':1, 'name':'park'})

2021-07-05 20:47:50,914 INFO sqlalchemy.engine.base.Engine INSERT INTO "T_users" (pk, name) VALUES (?, ?)
2021-07-05 20:47:50,916 INFO sqlalchemy.engine.base.Engine (1, 'park')
2021-07-05 20:47:50,926 INFO sqlalchemy.engine.base.Engine COMMIT


In [65]:
result = con.execute(obj_users.insert(), {'name':'lee'})

2021-07-05 20:48:21,412 INFO sqlalchemy.engine.base.Engine INSERT INTO "T_users" (name) VALUES (?)
2021-07-05 20:48:21,414 INFO sqlalchemy.engine.base.Engine ('lee',)
2021-07-05 20:48:21,423 INFO sqlalchemy.engine.base.Engine COMMIT


In [66]:
result.inserted_primary_key

[2]

In [67]:
from sqlalchemy import select

In [77]:
cur = con.execute(select([obj_users.c.name]))  

2021-07-05 20:51:59,455 INFO sqlalchemy.engine.base.Engine SELECT "T_users".name 
FROM "T_users"
2021-07-05 20:51:59,456 INFO sqlalchemy.engine.base.Engine ()


In [78]:
cur.fetchmany(1)

[('park',)]

In [79]:
cur.fetchmany(2)

[('lee',)]

In [116]:
print(obj_users.c.pk == obj_addresses.c.fk)

"T_users".pk = "T_addresses".fk


In [117]:
from sqlalchemy import and_, or_

In [118]:
print(and_(obj_users.c.pk == obj_addresses.c.fk, obj_users.c.pk ==1)) ## 두개의 조건을 한번에 사용하고 싶을 경우

"T_users".pk = "T_addresses".fk AND "T_users".pk = :pk_1


In [121]:
print((obj_users.c.pk == obj_addresses.c.fk) & (obj_users.c.pk ==1))

"T_users".pk = "T_addresses".fk AND "T_users".pk = :pk_1


In [127]:
print(select([obj_users]).where(obj_users.c.pk==1))

SELECT "T_users".pk, "T_users".name 
FROM "T_users" 
WHERE "T_users".pk = :pk_1


In [129]:
con.execute(obj_users.insert(), [{'name':'AA'},{'name':'BB'},{'name':'CC'}])

2021-07-05 17:20:06,239 INFO sqlalchemy.engine.base.Engine INSERT INTO "T_users" (name) VALUES (?)
2021-07-05 17:20:06,240 INFO sqlalchemy.engine.base.Engine (('AA',), ('BB',), ('CC',))
2021-07-05 17:20:06,249 INFO sqlalchemy.engine.base.Engine COMMIT


In [133]:
list(con.execute(select([obj_users]).where(obj_users.c.name.like('_A%'))))

2021-07-05 17:21:17,945 INFO sqlalchemy.engine.base.Engine SELECT "T_users".pk, "T_users".name 
FROM "T_users" 
WHERE "T_users".name LIKE ?
2021-07-05 17:21:17,947 INFO sqlalchemy.engine.base.Engine ('_A%',)


[(1, 'park'), (3, 'AA')]

In [134]:
from sqlalchemy import join

In [135]:
print(obj_users.join(obj_addresses))

"T_users" JOIN "T_addresses" ON "T_users".pk = "T_addresses".fk


In [137]:
print(select([obj_users]).select_from(obj_addresses))

SELECT "T_users".pk, "T_users".name 
FROM "T_users", "T_addresses"


In [139]:
print(select([obj_users]).select_from(obj_users.join(obj_addresses)))

SELECT "T_users".pk, "T_users".name 
FROM "T_users" JOIN "T_addresses" ON "T_users".pk = "T_addresses".fk


In [140]:
cur = con.execute(obj_addresses.insert({'name':'test','fk':1}))

2021-07-05 17:26:24,399 INFO sqlalchemy.engine.base.Engine INSERT INTO "T_addresses" (name, fk) VALUES (?, ?)
2021-07-05 17:26:24,400 INFO sqlalchemy.engine.base.Engine ('test', 1)
2021-07-05 17:26:24,409 INFO sqlalchemy.engine.base.Engine COMMIT


In [141]:
cur.inserted_primary_key

[1]